In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from PIL import Image
import cv2

In [2]:
data = pd.read_csv('Data/Facial expression recognition/Csv/fer2013.csv')

In [6]:
# Extract pixels and emotion labels
pixels = data['pixels'].tolist()
emotions = pd.get_dummies(data['emotion']).values

# Convert pixel values from string to numpy arrays
pixels = np.array([np.fromstring(pixel, dtype='int', sep=' ') for pixel in pixels])

# Filter out images with 48x48 dimensions
filtered_pixels = []
filtered_emotions = []
for image, emotion in zip(pixels, emotions):
    if image.size == 2304:  # Filter out images with 48x48 dimensions
        filtered_pixels.append(image)
        filtered_emotions.append(emotion)

filtered_pixels = np.array(filtered_pixels).reshape(-1, 48, 48, 1).astype('float32') / 255.0
filtered_emotions = np.array(filtered_emotions)

# Split the dataset into training, validation, and test sets
x_train, x_test, y_train, y_test = train_test_split(filtered_pixels, filtered_emotions, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.111, random_state=42)

In [7]:
# Data augmentation
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)

In [24]:
# Define the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=30, batch_size=64, callbacks=[early_stopping])

# save model and its architecture
model.save('Data/Facial expression recognition/Images/model.h5')

Epoch 1/30
449/449 [==============================] - 87s 191ms/step - loss: 1.6532 - accuracy: 0.3449 - val_loss: 1.4674 - val_accuracy: 0.4487
Epoch 2/30
449/449 [==============================] - 82s 183ms/step - loss: 1.4096 - accuracy: 0.4623 - val_loss: 1.3369 - val_accuracy: 0.5020
Epoch 3/30
449/449 [==============================] - 82s 182ms/step - loss: 1.2853 - accuracy: 0.5121 - val_loss: 1.2898 - val_accuracy: 0.5243
Epoch 4/30
449/449 [==============================] - 80s 179ms/step - loss: 1.1967 - accuracy: 0.5510 - val_loss: 1.1952 - val_accuracy: 0.5572
Epoch 5/30
449/449 [==============================] - 82s 182ms/step - loss: 1.1199 - accuracy: 0.5808 - val_loss: 1.1702 - val_accuracy: 0.5519
Epoch 6/30
449/449 [==============================] - 81s 180ms/step - loss: 1.0570 - accuracy: 0.6015 - val_loss: 1.1882 - val_accuracy: 0.5600
Epoch 7/30
449/449 [==============================] - 83s 185ms/step - loss: 0.9958 - accuracy: 0.6266 - val_loss: 1.1377 - val_ac

In [25]:
# load model
model = load_model('Data/Facial expression recognition/Images/model.h5')

# check model info
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 23, 23, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_29 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_29 (MaxPooli  (None, 10, 10, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_30 (Conv2D)          (None, 8, 8, 128)         73856     
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 4, 4, 128)        

In [26]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

113/113 [==============================] - 4s 29ms/step - loss: 1.2157 - accuracy: 0.5745
Test Loss: 1.2156680822372437
Test Accuracy: 0.5745332837104797


In [28]:
# Specify the directory where the images are located
image_directory = 'Data/Facial expression recognition/Images/test'

emotions_list = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
# Get a list of all emotion folders within the parent folder
emotion_folders = [f for f in os.listdir(image_directory) if os.path.isdir(os.path.join(image_directory, f))]

# Create a dictionary to store the counts of correct predictions and total images for each emotion
emotion_stats = {emotion: {'correct_count': 0, 'total_count': 0} for emotion in emotions_list}

# Loop through each emotion folder
for emotion_folder in emotion_folders:
    # Construct the full path to the emotion folder
    folder_path = os.path.join(image_directory, emotion_folder)

    # Get a list of all image files in the directory
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    total_count = len(image_files)

    # Preprocess and classify each test image
    for image_file in image_files:
        # Construct the full path to the image file
        image_path = os.path.join(folder_path, image_file)

        # Load and preprocess the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (48, 48))
        image = image.reshape(1, 48, 48, 1).astype('float32') / 255.0

        # Classify the image using the trained model
        predicted_emotion = model.predict(image)
        predicted_emotion = np.argmax(predicted_emotion)

        # Map the predicted label to the corresponding emotion
        predicted_emotion_label = emotions_list[predicted_emotion]

        # Update the counts for the corresponding emotion
        if emotion_folder.lower() == predicted_emotion_label.lower():
            emotion_stats[emotion_folder]['correct_count'] += 1
        emotion_stats[emotion_folder]['total_count'] += 1

        # Display the predicted emotion
        # print(f'Predicted Emotion for {image_path}: {predicted_emotion_label}')

# Calculate and display the accuracy rate for each emotion
for emotion, stats in emotion_stats.items():
    accuracy = stats['correct_count'] / stats['total_count'] * 100
    print(f'Accuracy for {emotion}: {accuracy:.2f}%')

# Calculate and display the total accuracy of the 7 emotions
total_correct = sum(stats['correct_count'] for stats in emotion_stats.values())
total_count = sum(stats['total_count'] for stats in emotion_stats.values())
total_accuracy = total_correct / total_count * 100
print(f'Total Accuracy: {total_accuracy:.2f}%')

1/1 [==============================] - 0s 62ms/step
Accuracy for angry: 67.12%
Accuracy for disgust: 45.05%
Accuracy for fear: 49.61%
Accuracy for happy: 88.50%
Accuracy for sad: 61.11%
Accuracy for surprise: 81.59%
Accuracy for neutral: 77.53%
Total Accuracy: 71.98%


In [2]:
# Set the training and validation data directories
train_data_dir = 'Data/Facial expression recognition/Images/train'
valid_data_dir = 'Data/Facial expression recognition/Images/validate'

# Set the parameters for data preprocessing and augmentation
batch_size = 64
image_size = (48, 48)

# Create the data generators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Get the list of class directories for both data sources
train_class_dirs = [os.path.join(train_data_dir, d) for d in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, d))]
valid_class_dirs = [os.path.join(valid_data_dir, d) for d in os.listdir(valid_data_dir) if os.path.isdir(os.path.join(valid_data_dir, d))]

# Collect the images and labels for both data sources
train_images = []
train_labels = []
for i, class_dir in enumerate(train_class_dirs):
    class_label = str(i)  # Convert the class label to string format
    for filename in os.listdir(class_dir):
        image_path = os.path.join(class_dir, filename)
        train_images.append(image_path)
        train_labels.append(class_label)

valid_images = []
valid_labels = []
for i, class_dir in enumerate(valid_class_dirs):
    class_label = str(i)  # Convert the class label to string format
    for filename in os.listdir(class_dir):
        image_path = os.path.join(class_dir, filename)
        valid_images.append(image_path)
        valid_labels.append(class_label)

# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
valid_images = np.array(valid_images)
valid_labels = np.array(valid_labels)

# Perform train-test split
train_images, test_images, train_labels, test_labels = train_test_split(np.concatenate((train_images, valid_images)), 
                                                                        np.concatenate((train_labels, valid_labels)), 
                                                                        test_size=0.2, random_state=42)

# Create data generators for training and validation data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": train_images, "class": train_labels}),
    directory=None,
    x_col="filename",
    y_col="class",
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": test_images, "class": test_labels}),
    directory=None,
    x_col="filename",
    y_col="class",
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

Found 27247 validated image filenames belonging to 7 classes.
Found 6812 validated image filenames belonging to 7 classes.


In [6]:
# Build a CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='selu', input_shape=(48, 48, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='selu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='selu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='selu'),
    layers.Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Set up early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model
epochs = 20
model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs,
    callbacks=[early_stopping]
)

# Evaluate the model on the validation data
validation_loss, validation_accuracy = model.evaluate(valid_generator)
print(f'Validation Loss: {validation_loss:.4f}')
print(f'Validation Accuracy: {validation_accuracy:.4f}')

# Save the trained model
model.save('Data/Facial expression recognition/Images/facial_expression_model_test_train.h5')

Epoch 1/20
426/426 [==============================] - 105s 241ms/step - loss: 1.8809 - accuracy: 0.2939 - val_loss: 1.8936 - val_accuracy: 0.2623
Epoch 2/20
426/426 [==============================] - 91s 213ms/step - loss: 1.6430 - accuracy: 0.3594 - val_loss: 1.4891 - val_accuracy: 0.4298
Epoch 3/20
426/426 [==============================] - 91s 214ms/step - loss: 1.5525 - accuracy: 0.4031 - val_loss: 1.5820 - val_accuracy: 0.4162
Epoch 4/20
426/426 [==============================] - 92s 217ms/step - loss: 1.4819 - accuracy: 0.4340 - val_loss: 1.3814 - val_accuracy: 0.4797
Epoch 5/20
426/426 [==============================] - 91s 212ms/step - loss: 1.4268 - accuracy: 0.4567 - val_loss: 1.3395 - val_accuracy: 0.4971
Epoch 6/20
426/426 [==============================] - 91s 213ms/step - loss: 1.3920 - accuracy: 0.4713 - val_loss: 1.3047 - val_accuracy: 0.5047
Epoch 7/20
426/426 [==============================] - 91s 214ms/step - loss: 1.3612 - accuracy: 0.4819 - val_loss: 1.3448 - val_a

In [2]:
# load model
model = load_model('Data/Facial expression recognition/Images/facial_expression_model_test_train.h5')

# check model info
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 46, 46, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 46, 46, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d_31 (MaxPooli  (None, 23, 23, 32)        0         
 ng2D)                                                           
                                                                 
 dropout (Dropout)           (None, 23, 23, 32)        0         
                                                                 
 conv2d_32 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 21, 21, 64)      

In [7]:
# Specify the directory where the images are located
image_directory = 'Data/Facial expression recognition/Images/test'

emotions_list = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
# Get a list of all emotion folders within the parent folder
emotion_folders = [f for f in os.listdir(image_directory) if os.path.isdir(os.path.join(image_directory, f))]

# Create a dictionary to store the counts of correct predictions and total images for each emotion
emotion_stats = {emotion: {'correct_count': 0, 'total_count': 0} for emotion in emotions_list}

# Loop through each emotion folder
for emotion_folder in emotion_folders:
    # Construct the full path to the emotion folder
    folder_path = os.path.join(image_directory, emotion_folder)

    # Get a list of all image files in the directory
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    total_count = len(image_files)

    # Preprocess and classify each test image
    for image_file in image_files:
        # Construct the full path to the image file
        image_path = os.path.join(folder_path, image_file)

        # Load and preprocess the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (48, 48))
        image = image.reshape(1, 48, 48, 1).astype('float32') / 255.0

        # Classify the image using the trained model
        predicted_emotion = model.predict(image)
        predicted_emotion = np.argmax(predicted_emotion)

        # Map the predicted label to the corresponding emotion
        predicted_emotion_label = emotions_list[predicted_emotion]

        # Update the counts for the corresponding emotion
        if emotion_folder.lower() == predicted_emotion_label.lower():
            emotion_stats[emotion_folder]['correct_count'] += 1
        emotion_stats[emotion_folder]['total_count'] += 1

        # Display the predicted emotion
        # print(f'Predicted Emotion for {image_path}: {predicted_emotion_label}')

# Calculate and display the accuracy rate for each emotion
for emotion, stats in emotion_stats.items():
    accuracy = stats['correct_count'] / stats['total_count'] * 100
    print(f'Accuracy for {emotion}: {accuracy:.2f}%')

# Calculate and display the total accuracy of the 7 emotions
total_correct = sum(stats['correct_count'] for stats in emotion_stats.values())
total_count = sum(stats['total_count'] for stats in emotion_stats.values())
total_accuracy = total_correct / total_count * 100
print(f'Total Accuracy: {total_accuracy:.2f}%')

1/1 [==============================] - 0s 29ms/step
Accuracy for angry: 49.79%
Accuracy for disgust: 23.42%
Accuracy for fear: 20.51%
Accuracy for happy: 86.19%
Accuracy for sad: 38.01%
Accuracy for surprise: 0.24%
Accuracy for neutral: 2.35%
Total Accuracy: 38.27%
